# thal2 3:2 Bifurcation Diagrams

# Inits

In [1]:
import sys; sys.path.insert(0, '..') # add parent folder path where lib folder is
# user-defined
import nmCoupling as nm
# import nmCoupling_old as nm
import response as rp

import thal2 as t2

from lib.util import (get_phase, freq_est, get_tongue_1d, pl_exist_1d,
get_period, bif1d, follow_phase_diffs, phase_lock_r3d, follow_locking_3d)

from lib.rhs import _redu_c,_full
from lib.rhs import _redu_3dc_thal as _redu_3dc

from lib.plot_util import (add_diagram_1d_scatter, draw_full_solutions, 
quick_plot_combined, quick_plot_r)

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import sympy as sym

from scipy.optimize import root,bisect
from copy import deepcopy
import os

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

nmc = nm.nmCoupling
rsp = rp.Response

## Dicts

In [2]:
kw_sim = {'rtol':1e-7,'atol':1e-7,'method':'LSODA'}

In [3]:
pd1 = {'gL':0.05,'gna':3,'gk':5,
       'gt':5,'eL':-70,'ena':50,
       'ek':-90,'et':0,'esyn':-1,
       'c':1,'alpha':3,'beta':2,
       'sigmat':0.8,'vt':-20,
       'ib':3.5,'del':0,'om':1,'om_fix':1}

In [4]:
# default period must be 2*np.pi
kws1 = {'var_names':['v','h','r','w'],
        'pardict':pd1,
        'rhs':t2.rhs,
        'coupling':t2.coupling,
        'init':np.array([-.64,0.71,0.25,0,6]),
        'TN':20000,
        'trunc_order':3,
        'z_forward':False,
        'i_forward':[False,True,False,False,False,False],
        'i_bad_dx':[False,True,False,False,False,False],
        'max_iter':50,
        'rtol':1e-12,
        'atol':1e-12,
        'rel_tol':1e-9,
        'save_fig':False,
        'lc_prominence':.05,
        'factor':.5}

## Defs

In [5]:
kw_bif = {'method':'LSODA','dense_output':True,'rtol':1e-9,'atol':1e-9}

In [6]:
etup23 = (.001,.1,100)
eps_list23r = np.linspace(*etup23)

In [7]:
x_temp = np.linspace(-np.pi,3*np.pi,500)

## Initialize Response Functions

In [8]:
system1 = rsp(idx=0,model_name='thal0_35',**kws1)
system2 = rsp(idx=1,model_name='thal1_35',**kws1)

mkdir ./data/thal0_35/
* Loading LC data...
self.T 6.283185307179586 omfix 1.6947246128737048
* Loading monodromy...
* Floquet Exponent kappa =-0.03594482780785962
* Loading g symbolic...
* Computing g...
* Loading heterogeneous terms...
* Computing z...
* Computing i...
mkdir ./data/thal1_35/
* Loading LC data...
self.T 6.283185307179586 omfix 1.6947246128737048
* Loading monodromy...
* Floquet Exponent kappa =-0.03594482780785962
* Loading g symbolic...
* Computing g...
* Loading heterogeneous terms...
* Computing z...
* Computing i...


In [9]:
kw_nm = {'system1':system1,'system2':system2,'_n':('om0',3),'_m':('om1',2),
        'NH':1024,'save_fig':False}

# del=0.0

In [10]:
del32_p0 = 0
a32_p0 = nm.nmCoupling(**kw_nm, del1=del32_p0)

pfactor 20
* Loading p symbolic...
* Loading p symbolic...
* Loading H symbolic...
* Loading H symbolic...
* Loading H thal0_35, order=0...
* Loading H thal1_35, order=0...
* Loading H thal0_35, order=1...
* Loading H thal1_35, order=1...
* Loading H thal0_35, order=2...
* Loading H thal1_35, order=2...
* Loading H thal0_35, order=3...
* Loading H thal1_35, order=3...


## Get Branches Full

In [11]:
kw_p0 = {'a':a32_p0,'del1':del32_p0,'recompute':True,
         'bifdir':'bif1d_thal2/','_full_rhs':_full}

In [12]:
data_p0_list = []

In [ ]:
in_dicts = [dict(init=2.5,eps_init=0.02,eps_final=0.11,deps=.01),
            dict(init=2.5,eps_init=0.02,eps_final=0.0005,deps=-.001)]

In [ ]:
for i in range(len(in_dicts)):
    in1 = in_dicts[i]
    dat = follow_phase_diffs(**in1,**kw_p0)
    data_p0_list.append(dat)
fname = kw_p0['bifdir']+'hyper_thal2_32_p0.txt'

In [ ]:
# save hyperparameters
hyp_inputs = np.zeros([len(in_dicts),4])
for i in range(len(in_dicts)):
    hyp_inputs[i,0] = in_dicts[i]['init']
    hyp_inputs[i,1] = in_dicts[i]['eps_init']
    hyp_inputs[i,2] = in_dicts[i]['eps_final']
    hyp_inputs[i,3] = in_dicts[i]['deps']
np.savetxt(fname,hyp_inputs)

## Get Branches 3D

In [ ]:
data_r3d_p0_list = []

In [ ]:
# find init for this branch mid
_,init = phase_lock_r3d([5,0,0],a32_p0,.05,_redu_3dc)
print(init)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p0,(.05,.057,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p0_list.append(dat)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p0,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p0_list.append(dat)

In [ ]:
# find init for branch 2
_,init = phase_lock_r3d([2.5,0,0],a32_p0,.05,_redu_3dc)
print(init)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p0,(.05,.062,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p0_list.append(dat)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p0,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p0_list.append(dat)

## Plot

In [ ]:
quick_plot_combined(a32_p0,
                    kw_f={'data_list':data_p0_list},
                    #kw_r3d={'data_list':data_r3d_p0_list},
                    kw_r={'etup':etup23})

# del=0.001

In [ ]:
del32_p001 = 0.001
a32_p001 = nm.nmCoupling(**kw_nm, del1=del32_p001)

## Get Branches Full

In [ ]:
kw_p001 = {'a':a32_p001,'del1':del32_p001,'recompute':True,
         'bifdir':'bif1d_thal2/','_full_rhs':_full}

In [ ]:
data_p001_list = []

In [ ]:
in_dicts = [dict(init=0,eps_init=0.02,eps_final=0.11,deps=.01),
            dict(init=0,eps_init=0.02,eps_final=0.0005,deps=-.001)]

In [ ]:
for i in range(len(in_dicts)):
    in1 = in_dicts[i]
    dat = follow_phase_diffs(**in1,**kw_p001)
    data_p001_list.append(dat)
fname = kw_p001['bifdir']+'hyper_thal2_32_p001.txt'

In [ ]:
# save hyperparameters
hyp_inputs = np.zeros([len(in_dicts),4])
for i in range(len(in_dicts)):
    hyp_inputs[i,0] = in_dicts[i]['init']
    hyp_inputs[i,1] = in_dicts[i]['eps_init']
    hyp_inputs[i,2] = in_dicts[i]['eps_final']
    hyp_inputs[i,3] = in_dicts[i]['deps']
np.savetxt(fname,hyp_inputs)

## Get Branches 3D

In [ ]:
data_r3d_p001_list = []

In [ ]:
# find init for this branch mid
_,init = phase_lock_r3d([5,0,0],a32_p001,.05,_redu_3dc)
print(init)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p001,(.05,.064,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p001_list.append(dat)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p001,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p001_list.append(dat)

In [ ]:
# find init for branch 2
_,init = phase_lock_r3d([2.5,0,0],a32_p001,.05,_redu_3dc)
print(init)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p001,(.05,.082,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p001_list.append(dat)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p001,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p001_list.append(dat)

In [ ]:
# find init for branch 3
_,init = phase_lock_r3d([.5,0,0],a32_p001,.05,_redu_3dc)
print(init)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p001,(.05,.082,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p001_list.append(dat)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p001,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p001_list.append(dat)

## Plot

In [ ]:
quick_plot_r(a32_p001,etup23)

In [ ]:
quick_plot_combined(a32_p001,
                    kw_f={'data_list':data_p001_list},
                    kw_r3d={'data_list':data_r3d_p001_list},
                    kw_r={'etup':etup23})

# del=0.003

In [ ]:
del32_p003 = 0.003
a32_p003 = nm.nmCoupling(**kw_nm, del1=del32_p003)

## Get Branches Full

In [ ]:
kw_p003 = {'a':a32_p003,'del1':del32_p003,'recompute':True,
         'bifdir':'bif1d_thal2/','_full_rhs':_full}

In [ ]:
data_p003_list = []

In [ ]:
in_dicts = [dict(init=0,eps_init=0.02,eps_final=0.11,deps=.01),
            dict(init=0,eps_init=0.01,eps_final=0.0005,deps=-.001)]

In [ ]:
for i in range(len(in_dicts)):
    in1 = in_dicts[i]
    dat = follow_phase_diffs(**in1,**kw_p003)
    data_p003_list.append(dat)
fname = kw_p003['bifdir']+'hyper_thal2_32_p003.txt'

In [ ]:
# save hyperparameters
hyp_inputs = np.zeros([len(in_dicts),4])
for i in range(len(in_dicts)):
    hyp_inputs[i,0] = in_dicts[i]['init']
    hyp_inputs[i,1] = in_dicts[i]['eps_init']
    hyp_inputs[i,2] = in_dicts[i]['eps_final']
    hyp_inputs[i,3] = in_dicts[i]['deps']
np.savetxt(fname,hyp_inputs)

## Get Branches 3D

In [ ]:
data_r3d_p003_list = []

In [ ]:
# find init for this branch mid
_,init = phase_lock_r3d([5,0,0],a32_p003,.05,_redu_3dc)
print(init)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p003,(.05,.11,.01),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p003_list.append(dat)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p003,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p003_list.append(dat)

In [ ]:
# find init for branch 2
_,init = phase_lock_r3d([2.5,0,0],a32_p003,.05,_redu_3dc)
print(init)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p003,(.05,.11,.01),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p003_list.append(dat)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p003,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p003_list.append(dat)

In [ ]:
# find init for branch 3
_,init = phase_lock_r3d([.5,0,0],a32_p003,.05,_redu_3dc)
print(init)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p003,(.05,.11,.01),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p003_list.append(dat)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p003,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p003_list.append(dat)

## Plot

In [ ]:
quick_plot_r(a32_p003,etup23)

In [ ]:
quick_plot_combined(a32_p003,
                    kw_f={'data_list':data_p003_list},
                    kw_r3d={'data_list':data_r3d_p003_list},
                    kw_r={'etup':etup23})

# del=0.004

In [ ]:
del32_p004 = 0.004
a32_p004 = nm.nmCoupling(**kw_nm, del1=del32_p004)

## Get Branches Full

In [ ]:
kw_p004 = {'a':a32_p004,'del1':del32_p004,'recompute':True,
         'bifdir':'bif1d_thal2/','_full_rhs':_full}

In [ ]:
data_p004_list = []

In [ ]:
in_dicts = [dict(init=0,eps_init=0.02,eps_final=0.11,deps=.01),
            dict(init=0,eps_init=0.01,eps_final=0.0005, deps=-.001)]

In [ ]:
for i in range(len(in_dicts)):
    in1 = in_dicts[i]
    dat = follow_phase_diffs(**in1,**kw_p004)
    data_p004_list.append(dat)
fname = kw_p004['bifdir']+'hyper_thal2_32_p004.txt'

In [ ]:
# save hyperparameters
hyp_inputs = np.zeros([len(in_dicts),4])
for i in range(len(in_dicts)):
    hyp_inputs[i,0] = in_dicts[i]['init']
    hyp_inputs[i,1] = in_dicts[i]['eps_init']
    hyp_inputs[i,2] = in_dicts[i]['eps_final']
    hyp_inputs[i,3] = in_dicts[i]['deps']
np.savetxt(fname,hyp_inputs)

## Get Branches 3D

In [ ]:
data_r3d_p004_list = []

In [ ]:
# find init for this branch mid
_,init = phase_lock_r3d([5,0,0],a32_p004,.05,_redu_3dc)
print(init)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p004,(.05,.11,.01),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p004_list.append(dat)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p004,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p004_list.append(dat)

In [ ]:
# find init for branch 2
_,init = phase_lock_r3d([2.5,0,0],a32_p004,.05,_redu_3dc)
print(init)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p004,(.05,.11,.01),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p004_list.append(dat)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p004,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p004_list.append(dat)

In [ ]:
# find init for branch 3
_,init = phase_lock_r3d([.5,0,0],a32_p004,.05,_redu_3dc)
print(init)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p004,(.05,.11,.01),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p004_list.append(dat)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p004,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p004_list.append(dat)

## Plot

In [ ]:
quick_plot_r(a32_p004,etup23)

In [ ]:
quick_plot_combined(a32_p004,
                    kw_f={'data_list':data_p004_list},
                    kw_r3d={'data_list':data_r3d_p004_list},
                    kw_r={'etup':etup23})

## Sims

In [ ]:
eps32_p004 = 0.05
th_init32_p004 = 3

In [ ]:
dt = .01;T_p004=2000
t32_p004 = np.arange(0,T_p004,dt)
args32_p004 = {'args':(a32_p004,eps32_p004,del32_p004),
               't_eval':t32_p004,**kw_sim}

In [ ]:
y0a = a32_p004.system1.lc['dat'][int((th_init32_p004/(2*np.pi)) * a32_p004.system1.TN),:]
y0b = a32_p004.system2.lc['dat'][int((0/(2*np.pi)) * a32_p004.system2.TN),:]

y0_p004 = np.array([y0a,y0b]).flatten()

solf32_p004 = solve_ivp(_full,[0,t32_p004[-1]],
                        y0_p004, **args32_p004)

In [ ]:
solr32_p004 = solve_ivp(_redu_c,[0,t32_p004[-1]],
                        [th_init32_p004],**args32_p004)

In [ ]:
# solr12b_p004 = solve_ivp(_redu_3dc,[0,t32_p004[-1]],
#                     [th_init32_p004,0,0],**args32_p004)

In [ ]:
tp12a_p004, phase12a_p004 = get_phase(t32_p004,solf32_p004.y.T[:,:4],
                            skipn=50,system1=a32_p004.system1)
tp12b_p004, phase12b_p004 = get_phase(t32_p004,solf32_p004.y.T[:,4:],
                            skipn=50,system1=a32_p004.system2)

In [ ]:
fig,axs = plt.subplots()
y = np.mod(phase12a_p004 - a32_p004.om*phase12b_p004,2*np.pi)
axs.scatter(tp12a_p004,y,s=1,alpha=.5,color='black')

axs.plot(t32_p004,np.mod(solr32_p004.y.T,2*np.pi),color='tab:red')

# axs.plot(t32_p004*a32_p004.om,np.mod(solr12b_p004.y[0]-a32_p004.om*solr12b_p004.y[2],2*np.pi),color='tab:blue')

#axs.plot(t12*a12.om,np.mod(solr12c.y[0]-a12.om*solr12c.y[2],2*np.pi),color='tab:green')
#axs.scatter(sol_full.t[::skipn],np.mod(phase2,1),s=1)

axs.set_ylim(-.5,2*np.pi+.5)

# del=0.005

In [ ]:
del32_p005 = 0.005
a32_p005 = nm.nmCoupling(**kw_nm, del1=del32_p005)

## Get Branches Full

In [ ]:
kw_p005 = {'a':a32_p005,'del1':del32_p005,'recompute':True,
         'bifdir':'bif1d_thal2/','_full_rhs':_full}

In [ ]:
data_p005_list = []

In [ ]:
in_dicts = [dict(init=0,eps_init=0.01,eps_final=0.11,deps=.01),
            dict(init=0,eps_init=0.01,eps_final=0.0005,deps=-.001)]

In [ ]:
for i in range(len(in_dicts)):
    in1 = in_dicts[i]
    dat = follow_phase_diffs(**in1,**kw_p005)
    data_p005_list.append(dat)
fname = kw_p005['bifdir']+'hyper_thal2_32_p005.txt'

In [ ]:
# save hyperparameters
hyp_inputs = np.zeros([len(in_dicts),4])
for i in range(len(in_dicts)):
    hyp_inputs[i,0] = in_dicts[i]['init']
    hyp_inputs[i,1] = in_dicts[i]['eps_init']
    hyp_inputs[i,2] = in_dicts[i]['eps_final']
    hyp_inputs[i,3] = in_dicts[i]['deps']
np.savetxt(fname,hyp_inputs)

## Get Branches 3D

In [ ]:
data_r3d_p005_list = []

In [ ]:
# find init for this branch mid
_,init = phase_lock_r3d([5,0,0],a32_p005,.05,_redu_3dc)
print(init)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p005,(.05,.064,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p005_list.append(dat)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p005,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p005_list.append(dat)

In [ ]:
# find init for branch 2
_,init = phase_lock_r3d([2.5,0,0],a32_p005,.05,_redu_3dc)
print(init)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p005,(.05,.082,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p005_list.append(dat)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p005,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p005_list.append(dat)

In [ ]:
# find init for branch 3
_,init = phase_lock_r3d([.5,0,0],a32_p005,.05,_redu_3dc)
print(init)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p005,(.05,.082,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p005_list.append(dat)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p005,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p005_list.append(dat)

## Plot

In [ ]:
quick_plot_r(a32_p005,etup23)

In [ ]:
quick_plot_combined(a32_p005,
                    kw_f={'data_list':data_p005_list},
                    kw_r3d={'data_list':data_r3d_p005_list},
                    kw_r={'etup':etup23})

# del=0.007

In [ ]:
del32_p007 = 0.007
a32_p007 = nm.nmCoupling(**kw_nm, del1=del32_p007)

## Get Branches Full

In [ ]:
kw_p007 = {'a':a32_p007,'del1':del32_p007,'recompute':True,
         'bifdir':'bif1d_thal2/','_full_rhs':_full}

In [ ]:
data_p007_list = []

In [ ]:
in_dicts = [dict(init=0,eps_init=0.01,eps_final=0.11,deps=.01),
            dict(init=0,eps_init=0.01,eps_final=0.0005,deps=-.001)]

In [ ]:
for i in range(len(in_dicts)):
    in1 = in_dicts[i]
    dat = follow_phase_diffs(**in1,**kw_p007)
    data_p007_list.append(dat)
fname = kw_p007['bifdir']+'hyper_thal2_32_p007.txt'

In [ ]:
# save hyperparameters
hyp_inputs = np.zeros([len(in_dicts),4])
for i in range(len(in_dicts)):
    hyp_inputs[i,0] = in_dicts[i]['init']
    hyp_inputs[i,1] = in_dicts[i]['eps_init']
    hyp_inputs[i,2] = in_dicts[i]['eps_final']
    hyp_inputs[i,3] = in_dicts[i]['deps']
np.savetxt(fname,hyp_inputs)

## Get Branches 3D

In [ ]:
data_r3d_p007_list = []

In [ ]:
# find init for this branch mid
_,init = phase_lock_r3d([5,0,0],a32_p007,.05,_redu_3dc)
print(init)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p007,(.05,.064,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p007_list.append(dat)

In [ ]:
# branch 1
dat = follow_locking_3d(init,a32_p007,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p007_list.append(dat)

In [ ]:
# find init for branch 2
_,init = phase_lock_r3d([2.5,0,0],a32_p007,.05,_redu_3dc)
print(init)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p007,(.05,.082,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p007_list.append(dat)

In [ ]:
# branch 2
dat = follow_locking_3d(init,a32_p007,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p007_list.append(dat)

In [ ]:
# find init for branch 3
_,init = phase_lock_r3d([.5,0,0],a32_p007,.05,_redu_3dc)
print(init)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p007,(.05,.082,.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p007_list.append(dat)

In [ ]:
# branch 3
dat = follow_locking_3d(init,a32_p007,(.05,.001,-.002),
                        rhs=_redu_3dc,recompute=False)
data_r3d_p007_list.append(dat)

## Plot

In [ ]:
quick_plot_r(a32_p007,etup23)

In [ ]:
quick_plot_combined(a32_p007,
                    kw_f={'data_list':data_p007_list},
                    kw_r3d={'data_list':data_r3d_p007_list},
                    kw_r={'etup':etup23})